In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.engine.topology import Layer
from keras import backend as K
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from utils import LRN2D
import utils

%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Contruct the model structure

In [19]:
myInput = Input(shape=(3, 96, 96))

x = ZeroPadding2D(padding=(3, 3), input_shape=(3, 96, 96), data_format='channels_first')(myInput)
x = Conv2D(64, (7, 7), strides=(2, 2), data_format='channels_first', name='conv1')(x)
x = BatchNormalization(axis=1, epsilon=0.00001, name='bn1')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1), data_format='channels_first')(x)
x = MaxPooling2D(pool_size=3, strides=2, data_format='channels_first')(x)
x = Conv2D(64, (1, 1), data_format='channels_first', name='conv2')(x)
x = BatchNormalization(axis=1, epsilon=0.00001, name='bn2')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1), data_format='channels_first')(x)
x = Conv2D(192, (3, 3), data_format='channels_first', name='conv3')(x)
x = BatchNormalization(axis=1, epsilon=0.00001, name='bn3')(x)
x = Activation('relu')(x)
x = ZeroPadding2D(padding=(1, 1), data_format='channels_first')(x)
x = MaxPooling2D(pool_size=3, strides=2, data_format='channels_first')(x)

# Inception3a
inception_3a_3x3 = Conv2D(96, (1, 1), data_format='channels_first', name='inception_3a_3x3_conv1')(x)
inception_3a_3x3 = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3a_3x3_bn1')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)
inception_3a_3x3 = ZeroPadding2D(padding=(1, 1), data_format='channels_first')(inception_3a_3x3)
inception_3a_3x3 = Conv2D(128, (3, 3), data_format='channels_first', name='inception_3a_3x3_conv2')(inception_3a_3x3)
inception_3a_3x3 = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3a_3x3_bn2')(inception_3a_3x3)
inception_3a_3x3 = Activation('relu')(inception_3a_3x3)

inception_3a_5x5 = Conv2D(16, (1, 1), data_format='channels_first', name='inception_3a_5x5_conv1')(x)
inception_3a_5x5 = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3a_5x5_bn1')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)
inception_3a_5x5 = ZeroPadding2D(padding=(2, 2), data_format='channels_first')(inception_3a_5x5)
inception_3a_5x5 = Conv2D(32, (5, 5), data_format='channels_first', name='inception_3a_5x5_conv2')(inception_3a_5x5)
inception_3a_5x5 = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3a_5x5_bn2')(inception_3a_5x5)
inception_3a_5x5 = Activation('relu')(inception_3a_5x5)

inception_3a_pool = MaxPooling2D(pool_size=3, strides=2, data_format='channels_first')(x)
inception_3a_pool = Conv2D(32, (1, 1), data_format='channels_first', name='inception_3a_pool_conv')(inception_3a_pool)
inception_3a_pool = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3a_pool_bn')(inception_3a_pool)
inception_3a_pool = Activation('relu')(inception_3a_pool)
inception_3a_pool = ZeroPadding2D(padding=((3, 4), (3, 4)), data_format='channels_first')(inception_3a_pool)

inception_3a_1x1 = Conv2D(64, (1, 1), data_format='channels_first', name='inception_3a_1x1_conv')(x)
inception_3a_1x1 = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3a_1x1_bn')(inception_3a_1x1)
inception_3a_1x1 = Activation('relu')(inception_3a_1x1)

inception_3a = concatenate([inception_3a_3x3, inception_3a_5x5, inception_3a_pool, inception_3a_1x1], axis=1)

# Inception3b
inception_3b_3x3 = Conv2D(96, (1, 1), data_format='channels_first', name='inception_3b_3x3_conv1')(inception_3a)
inception_3b_3x3 = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3b_3x3_bn1')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)
inception_3b_3x3 = ZeroPadding2D(padding=(1, 1), data_format='channels_first')(inception_3b_3x3)
inception_3b_3x3 = Conv2D(128, (3, 3), data_format='channels_first', name='inception_3b_3x3_conv2')(inception_3b_3x3)
inception_3b_3x3 = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3b_3x3_bn2')(inception_3b_3x3)
inception_3b_3x3 = Activation('relu')(inception_3b_3x3)

inception_3b_5x5 = Conv2D(32, (1, 1), data_format='channels_first', name='inception_3b_5x5_conv1')(inception_3a)
inception_3b_5x5 = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3b_5x5_bn1')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)
inception_3b_5x5 = ZeroPadding2D(padding=(2, 2), data_format='channels_first')(inception_3b_5x5)
inception_3b_5x5 = Conv2D(64, (5, 5), data_format='channels_first', name='inception_3b_5x5_conv2')(inception_3b_5x5)
inception_3b_5x5 = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3b_5x5_bn2')(inception_3b_5x5)
inception_3b_5x5 = Activation('relu')(inception_3b_5x5)

# inception_3b_pool = inception_3a
# inception_3b_pool = Lambda(lambda x: x**2)(inception_3a)
inception_3b_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3), data_format='channels_first')(inception_3a)
# inception_3b_pool = Lambda(lambda x: x*9)(inception_3b_pool)
# inception_3b_pool = Lambda(lambda x: K.sqrt(x))(inception_3b_pool)
inception_3b_pool = Conv2D(64, (1, 1), data_format='channels_first', name='inception_3b_pool_conv')(inception_3b_pool)
inception_3b_pool = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3b_pool_bn')(inception_3b_pool)
inception_3b_pool = Activation('relu')(inception_3b_pool)
inception_3b_pool = ZeroPadding2D(padding=(4, 4), data_format='channels_first')(inception_3b_pool)

inception_3b_1x1 = Conv2D(64, (1, 1), data_format='channels_first', name='inception_3b_1x1_conv')(inception_3a)
inception_3b_1x1 = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3b_1x1_bn')(inception_3b_1x1)
inception_3b_1x1 = Activation('relu')(inception_3b_1x1)

inception_3b = concatenate([inception_3b_3x3, inception_3b_5x5, inception_3b_pool, inception_3b_1x1], axis=1)

# Inception3c
inception_3c_3x3 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_3x3',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))

inception_3c_5x5 = utils.conv2d_bn(inception_3b,
                                   layer='inception_3c_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))

inception_3c_pool = MaxPooling2D(pool_size=3, strides=2, data_format='channels_first')(inception_3b)
inception_3c_pool = ZeroPadding2D(padding=((0, 1), (0, 1)), data_format='channels_first')(inception_3c_pool)

inception_3c = concatenate([inception_3c_3x3, inception_3c_5x5, inception_3c_pool], axis=1)

#inception 4a
inception_4a_3x3 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=192,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_4a_5x5 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_5x5',
                                   cv1_out=32,
                                   cv1_filter=(1, 1),
                                   cv2_out=64,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(1, 1),
                                   padding=(2, 2))
# inception_4a_pool = inception_3c
# inception_4a_pool = Lambda(lambda x: x**2)(inception_3c)
inception_4a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3), data_format='channels_first')(inception_3c)
# inception_4a_pool = Lambda(lambda x: x*9)(inception_4a_pool)
# inception_4a_pool = Lambda(lambda x: K.sqrt(x))(inception_4a_pool)
inception_4a_pool = utils.conv2d_bn(inception_4a_pool,
                                   layer='inception_4a_pool',
                                   cv1_out=128,
                                   cv1_filter=(1, 1),
                                   padding=(2, 2))
inception_4a_1x1 = utils.conv2d_bn(inception_3c,
                                   layer='inception_4a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_4a = concatenate([inception_4a_3x3, inception_4a_5x5, inception_4a_pool, inception_4a_1x1], axis=1)

#inception4e
inception_4e_3x3 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_3x3',
                                   cv1_out=160,
                                   cv1_filter=(1, 1),
                                   cv2_out=256,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(2, 2),
                                   padding=(1, 1))
inception_4e_5x5 = utils.conv2d_bn(inception_4a,
                                   layer='inception_4e_5x5',
                                   cv1_out=64,
                                   cv1_filter=(1, 1),
                                   cv2_out=128,
                                   cv2_filter=(5, 5),
                                   cv2_strides=(2, 2),
                                   padding=(2, 2))
inception_4e_pool = MaxPooling2D(pool_size=3, strides=2, data_format='channels_first')(inception_4a)
inception_4e_pool = ZeroPadding2D(padding=((0, 1), (0, 1)), data_format='channels_first')(inception_4e_pool)

inception_4e = concatenate([inception_4e_3x3, inception_4e_5x5, inception_4e_pool], axis=1)

#inception5a
inception_5a_3x3 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
# inception_5a_pool = inception_4e
# inception_5a_pool = Lambda(lambda x: x**2)(inception_4e)
inception_5a_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3), data_format='channels_first')(inception_4e)
# inception_5a_pool = Lambda(lambda x: x*9)(inception_5a_pool)
# inception_5a_pool = Lambda(lambda x: K.sqrt(x))(inception_5a_pool)
inception_5a_pool = utils.conv2d_bn(inception_5a_pool,
                                   layer='inception_5a_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   padding=(1, 1))
inception_5a_1x1 = utils.conv2d_bn(inception_4e,
                                   layer='inception_5a_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))

inception_5a = concatenate([inception_5a_3x3, inception_5a_pool, inception_5a_1x1], axis=1)

#inception_5b
inception_5b_3x3 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_3x3',
                                   cv1_out=96,
                                   cv1_filter=(1, 1),
                                   cv2_out=384,
                                   cv2_filter=(3, 3),
                                   cv2_strides=(1, 1),
                                   padding=(1, 1))
inception_5b_pool = MaxPooling2D(pool_size=3, strides=2, data_format='channels_first')(inception_5a)
inception_5b_pool = utils.conv2d_bn(inception_5b_pool,
                                   layer='inception_5b_pool',
                                   cv1_out=96,
                                   cv1_filter=(1, 1))
inception_5b_pool = ZeroPadding2D(padding=(1, 1), data_format='channels_first')(inception_5b_pool)

inception_5b_1x1 = utils.conv2d_bn(inception_5a,
                                   layer='inception_5b_1x1',
                                   cv1_out=256,
                                   cv1_filter=(1, 1))
inception_5b = concatenate([inception_5b_3x3, inception_5b_pool, inception_5b_1x1], axis=1)

av_pool = AveragePooling2D(pool_size=(3, 3), strides=(1, 1), data_format='channels_first')(inception_5b)
reshape_layer = Flatten()(av_pool)
dense_layer = Dense(128, name='dense_layer')(reshape_layer)
# norm_layer = K.l2_normalize(dense_layer, axis=1)
norm_layer = Lambda(lambda  x: K.l2_normalize(x,axis=1))(dense_layer)


# Final Model
newModel = Model(inputs=[myInput], outputs=dense_layer)


## Load weights, Set weights and Save the model

In [20]:
# Load weights from csv files (which was exported from Openface torch model)
weights = utils.weights
weights_dict = utils.load_weights()

# Set layer weights of the model
for name in weights:
  if newModel.get_layer(name) != None:
    newModel.get_layer(name).set_weights(weights_dict[name])
  elif model.get_layer(name) != None:
    model.get_layer(name).set_weights(weights_dict[name])

# Save Keras model into HDF5 file
# newModel.save('./model/nn4.small2.v1.h5')

In [16]:
newModel.save('./model/nn4.small2.v7.h5')

In [ ]:
!open .

## Generate the embeddings 

In [21]:
# Read a sample image as input to test the model
img = cv2.imread('./data/dlib-affine-sz/Aaron_Eckhart/Aaron_Eckhart_0001.png', 1)
img = img[...,::-1]
img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)

x_train = np.array([img])
y = newModel.predict_on_batch(x_train)
print y

[[-0.39416081 -0.39163673  0.02796706 -0.40874565 -0.26105183 -0.12685221
  -1.09747362  0.282323   -0.34551656 -0.57799441  0.14906463 -0.28493878
  -0.08615411  1.18118322  0.99771112 -0.41728529  0.55623996 -1.12885964
   1.57037759 -0.15986727 -0.41109258  0.55860186  0.60203791 -0.84212905
   0.12272055  0.53843957  0.33578238 -0.40378648 -0.73423457 -0.65141034
   0.79520911 -1.40462291  0.75667298 -0.63002127 -0.0713354   1.26806211
   0.36766914  0.37059116  0.35777858  1.38767242 -0.26460516  1.30055296
  -1.04249167 -0.49836773 -0.20425759  0.94612569  0.19634613 -1.19779205
   0.38501868 -0.44128111 -0.97510856 -0.0179598   0.17234439  0.37226856
  -0.71182114  0.190319    0.44335291  0.2156883  -0.42035928  0.85428989
  -0.18945013  0.14354174 -0.31195003 -0.74808723  0.62867481 -0.81627995
   0.98924452  1.00418973  1.2318449   0.10502893  1.08896101  0.35668638
  -0.19775483 -0.61878431  0.6862542   0.57292885 -1.16660535  0.09685978
  -0.27057776  0.34014243 -1.16521728 

In [22]:
# LFW TEST
import lfw
import os
import numpy as np
import math
import facenet
import time
import tensorflow as tf

lfw_pairs='data/pairs.txt'
lfw_dir='data/dlib-affine-sz'
lfw_file_ext='png'
lfw_nrof_folds=10
image_size=96
batch_size=100

# Read the file containing the pairs used for testing
pairs = lfw.read_pairs(os.path.expanduser(lfw_pairs))

# Get the paths for the corresponding images
paths, actual_issame = lfw.get_paths(os.path.expanduser(lfw_dir), pairs, lfw_file_ext)

embedding_size=128
nrof_images = len(paths)
nrof_batches = int(math.ceil(1.0*nrof_images / batch_size))
emb_array = np.zeros((nrof_images, embedding_size))

# print paths

for i in range(nrof_batches):
  start_index = i*batch_size
  end_index = min((i+1)*batch_size, nrof_images)
  paths_batch = paths[start_index:end_index]
  images = facenet.load_data(paths_batch, False, False, image_size)
  images = np.transpose(images, (0,3,1,2))
  
  t0 = time.time()
  y = newModel.predict_on_batch(images)
  emb_array[start_index:end_index,:] = y
#   print('y', y)
#   print('emb', emb_array[start_index:end_index,:])
  t1 = time.time()
  
  print('batch: ', i, ' time: ', t1-t0)

from sklearn import metrics
from scipy.optimize import brentq
from scipy import interpolate

tpr, fpr, accuracy, val, val_std, far = lfw.evaluate(emb_array, 
                actual_issame, nrof_folds=lfw_nrof_folds)

print('Accuracy: %1.3f+-%1.3f' % (np.mean(accuracy), np.std(accuracy)))
print('Validation rate: %2.5f+-%2.5f @ FAR=%2.5f' % (val, val_std, far))
auc = metrics.auc(fpr, tpr)
print('Area Under Curve (AUC): %1.3f' % auc)

eer = brentq(lambda x: 1. - x - interpolate.interp1d(fpr, tpr)(x), 0., 1.)
print('Equal Error Rate (EER): %1.3f' % eer)

Skipped 45 image pairs
('batch: ', 0, ' time: ', 15.178622007369995)
('batch: ', 1, ' time: ', 15.061661005020142)
('batch: ', 2, ' time: ', 18.344292163848877)
('batch: ', 3, ' time: ', 14.834142923355103)
('batch: ', 4, ' time: ', 15.600435018539429)
('batch: ', 5, ' time: ', 15.668452978134155)
('batch: ', 6, ' time: ', 14.760615110397339)
('batch: ', 7, ' time: ', 13.961729049682617)
('batch: ', 8, ' time: ', 19.42907214164734)
('batch: ', 9, ' time: ', 15.307490110397339)
('batch: ', 10, ' time: ', 17.867768049240112)
('batch: ', 11, ' time: ', 15.594386100769043)
('batch: ', 12, ' time: ', 16.41095495223999)
('batch: ', 13, ' time: ', 18.49483609199524)
('batch: ', 14, ' time: ', 13.538466930389404)
('batch: ', 15, ' time: ', 14.66977310180664)
('batch: ', 16, ' time: ', 14.098972082138062)
('batch: ', 17, ' time: ', 15.781857013702393)
('batch: ', 18, ' time: ', 12.949981927871704)
('batch: ', 19, ' time: ', 13.16347885131836)
('batch: ', 20, ' time: ', 12.822646141052246)
('bat

ValueError: A value in x_new is above the interpolation range.

In [6]:
# emb_array = np.genfromtxt("/Users/victor_sy_wang/Developer/ML/keras-facenet/emb_array.csv", delimiter=",")
from sklearn import metrics
from scipy.optimize import brentq
from scipy import interpolate

tpr, fpr, accuracy, val, val_std, far = lfw.evaluate(emb_array, 
                actual_issame, nrof_folds=lfw_nrof_folds)

print('Accuracy: %1.3f+-%1.3f' % (np.mean(accuracy), np.std(accuracy)))
# print('Validation rate: %2.5f+-%2.5f @ FAR=%2.5f' % (val, val_std, far))
# auc = metrics.auc(fpr, tpr)
# print('Area Under Curve (AUC): %1.3f' % auc)
# eer = brentq(lambda x: 1. - x - interpolate.interp1d(fpr, tpr)(x), 0., 1.)
# print('Equal Error Rate (EER): %1.3f' % eer)

Accuracy: 0.501+-0.003


In [19]:
np.savetxt("emb_array.csv", emb_array, delimiter=",")

e